In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
dataFolder = '/home/santhosr/Documents/Birad/ProcessedData/FullRes'
newDataFolder = '/home/santhosr/Documents/Birad/ProcessedData/FullRes_RaceSampled'

In [3]:
classFolders = os.listdir(dataFolder)

#FastAi tends to add 'models' folder in the data directory
if 'models' in classFolders:
    classFolders.remove('models')

In [4]:
classFolders

['3', '4', '2', '1']

In [5]:
## Number of images to have in each folder
numImages = 1000

In [6]:
#Creating folders

os.chdir(newDataFolder)

for i in classFolders:
    !mkdir $i

### Basic Reshuffling

In [56]:
for i in range(len(classFolders)):
    
    label = classFolders[i]
    
    fileList = os.listdir(os.path.join(dataFolder,label))
    numFiles = len(fileList)
    print(label)
    print(numFiles)
    
    if numFiles>numImages:
        
        selectedFiles = np.random.choice(fileList,numImages, replace=False)
        
        for j in range(numImages):
            inputFile = os.path.join(dataFolder , label, selectedFiles[j])
            outputFile = os.path.join(newDataFolder , label, selectedFiles[j])
            ! ln -sf $inputFile $outputFile
            
    else:
        
        for j in range(numFiles):
            inputFile = os.path.join(dataFolder , label, fileList[j])
            outputFile = os.path.join(newDataFolder , label, fileList[j])
            ! ln -sf $inputFile $outputFile
            
        selectedFiles = np.random.choice(fileList,numImages - numFiles, replace=True)
        
        for j in range(numImages-numFiles):
            inputFile = os.path.join(dataFolder , label, selectedFiles[j])
            outputFile = os.path.join(newDataFolder , label, str(int(np.random.random()*100000))+"_"+selectedFiles[j])
            ! ln -sf $inputFile $outputFile
        

3
1403
4
112
2
3445
1
964


In [20]:
a = os.listdir('/home/santhosr/Documents/Birad/ProcessedData/FullRes_Sampled/4')

In [21]:
len(a)

112

In [19]:
2250+560

2810

### Getting Patient List

In [8]:
truthFile = '/home/santhosr/Documents/Birad/birad_targetFile.csv'
truth = pd.read_csv(truthFile)
truth.head()

,BMI,Medview_Race,ScreenDate,Age,DummyID,PresIntentType,DBT,Density_Overall
0,27.1,White,3/23/2011,67.8,2508626,FOR PROCESSING,NaN,2
1,25.7,White,6/2/2011,58.8,2505738,FOR PROCESSING,NaN,2
2,28.1,White,5/24/2011,56.4,2547962,FOR PROCESSING,NaN,2
3,31.8,White,5/6/2011,51.7,2575490,FOR PROCESSING,NaN,2
4,27.5,Other,5/5/2011,63.7,2591080,FOR PROCESSING\,No Tomo images,2


In [11]:
fullPatientList = []
patientList = {'African American':[], 'White':[]}

for i in range(len(classFolders)):
    
    label = classFolders[i]
    
    fileList = os.listdir(os.path.join(dataFolder,label))
    
    for j in range(len(fileList)):
        
        patient = fileList[j].split("_")[0]
        
        if patient not in fullPatientList:
            
            fullPatientList.append(patient)
            
            try:
                race = truth.loc[truth.DummyID == int(patient)]['Medview_Race'].values[0]
            except:
                print(patient)
            
            if race in ['African American','White']:
                patientList[race].append(patient) 


In [20]:
len(patientList['African American'])

1428

### Reshuffling with Race

In [21]:
whiteList = np.random.choice(patientList['White'],600, replace=False)
AAlist = np.random.choice(patientList['African American'],600, replace=False)

In [23]:
def cleanFolderList(fileList, patientList):
    """
    Given a list of patient files, it retains only those files that are from patients given in the patientList
    """
    
    cleanFileList = []
    
    for i in range(len(fileList)):
        
        patient = fileList[i].split("_")[0]
        
        if patient in patientList['White'] or patient in patientList['African American']:
            cleanFileList.append(fileList[i])
            
    return cleanFileList

In [25]:
for i in range(len(classFolders)):
    
    label = classFolders[i]
    
    print(label)
    fileList = os.listdir(os.path.join(dataFolder,label))
    print(len(fileList))
    fileList = cleanFolderList(fileList, patientList)
    numFiles = len(fileList)
    
    print(numFiles)
    
    if numFiles>numImages:
        
        selectedFiles = np.random.choice(fileList,numImages, replace=False)
        
        for j in range(numImages):
            inputFile = os.path.join(dataFolder , label, selectedFiles[j])
            outputFile = os.path.join(newDataFolder , label, selectedFiles[j])
            ! ln -sf $inputFile $outputFile
            
    else:
        
        for j in range(numFiles):
            inputFile = os.path.join(dataFolder , label, fileList[j])
            outputFile = os.path.join(newDataFolder , label, fileList[j])
            ! ln -sf $inputFile $outputFile
            
        selectedFiles = np.random.choice(fileList,numImages - numFiles, replace=True)
        
        for j in range(numImages-numFiles):
            inputFile = os.path.join(dataFolder , label, selectedFiles[j])
            outputFile = os.path.join(newDataFolder , label, str(int(np.random.random()*100000))+"_"+selectedFiles[j])
            ! ln -sf $inputFile $outputFile
        

3
1991
1872
4
134
118
2
5279
5104
1
1344
1324


In [24]:
newDataFolder

'/home/santhosr/Documents/Birad/ProcessedData/FullRes_RaceSampled'